<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    48
1    13
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1060.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,,
2022-12-06,18643.0,64.0,^NSEI,7.45,0.77,0.83,18812.0,15294.0,95.19,4.81


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# heavy upside
df_prec_dev.sort_values(by = 'Upside%', ascending=False).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,668.0,40.0,MOTILALOFS.NS,-13.71,-1.55,-1.01,980.0,654.0,4.26,95.74,930.0,39.22
1,2022-12-06,465.0,73.0,GODREJIND.NS,-0.51,-0.64,1.15,644.0,405.0,24.94,75.06,638.0,37.20
2,2022-12-06,468.0,40.0,ICICIPRULI.NS,-11.20,-1.14,-1.00,616.0,447.0,12.43,87.57,640.0,36.75
3,2022-12-06,1285.0,60.0,SBILIFE.NS,6.38,0.50,0.56,1332.0,1025.0,84.72,15.28,1604.0,24.82
4,2022-12-06,6699.0,37.0,BAJFINANCE.NS,-2.33,-0.20,-0.85,7853.0,5284.0,55.06,44.94,8015.0,19.64
5,2022-12-06,923.0,51.0,ICICIBANK.NS,11.43,1.33,0.66,953.0,654.0,89.85,10.15,1094.0,18.53
6,2022-12-06,1184.0,55.0,INDUSINDBK.NS,13.09,1.34,0.43,1264.0,778.0,83.56,16.44,1393.0,17.65
7,2022-12-06,905.0,68.0,AXISBANK.NS,17.95,1.69,1.42,915.0,626.0,96.42,3.58,1060.0,17.13
8,2022-12-06,3300.0,34.0,EICHERMOT.NS,5.15,0.86,-0.69,3850.0,2256.0,65.48,34.52,3865.0,17.12
9,2022-12-06,555.0,44.0,JUBLFOOD.NS,-6.36,-0.67,-1.05,786.0,464.0,28.35,71.65,649.0,16.94


In [12]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,668.0,40.0,MOTILALOFS.NS,-13.71,-1.55,-1.01,980.0,654.0,4.26,95.74,930.0,39.22
35,2022-12-06,3356.0,43.0,DIVISLAB.NS,-11.76,-1.24,-0.60,4678.0,3266.0,6.37,93.63,3560.0,6.08
2,2022-12-06,468.0,40.0,ICICIPRULI.NS,-11.20,-1.14,-1.00,616.0,447.0,12.43,87.57,640.0,36.75
44,2022-12-06,411.0,65.0,WIPRO.NS,-9.59,-1.10,0.50,722.0,375.0,10.48,89.52,425.0,3.41
9,2022-12-06,555.0,44.0,JUBLFOOD.NS,-6.36,-0.67,-1.05,786.0,464.0,28.35,71.65,649.0,16.94
30,2022-12-06,1089.0,57.0,TECHM.NS,-6.30,-0.69,0.66,1806.0,965.0,14.72,85.28,1158.0,6.34
15,2022-12-06,78.0,60.0,NATIONALUM.NS,-5.54,-1.20,1.33,130.0,67.0,18.10,81.90,90.0,15.38
14,2022-12-06,4053.0,56.0,NAUKRI.NS,-4.34,-0.64,0.33,5740.0,3375.0,28.67,71.33,4677.0,15.40
40,2022-12-06,618.0,54.0,BERGEPAINT.NS,-3.66,-0.47,0.16,791.0,561.0,25.00,75.00,651.0,5.34
25,2022-12-06,1222.0,45.0,HAVELLS.NS,-2.90,-0.16,-0.16,1418.0,1070.0,43.81,56.19,1328.0,8.67


In [13]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
55,2022-12-06,319.0,62.0,CANBK.NS,30.59,3.28,2.43,329.0,175.0,93.28,6.72,310.0,-2.82
21,2022-12-06,135.0,55.0,FEDERALBNK.NS,20.36,1.79,0.78,138.0,81.0,94.39,5.61,149.0,10.37
58,2022-12-06,4424.0,78.0,BRITANNIA.NS,18.65,1.64,2.09,4433.0,3094.0,99.35,0.65,3935.0,-11.05
7,2022-12-06,905.0,68.0,AXISBANK.NS,17.95,1.69,1.42,915.0,626.0,96.42,3.58,1060.0,17.13
10,2022-12-06,609.0,59.0,SBIN.NS,14.90,1.51,1.03,617.0,435.0,95.58,4.42,710.0,16.58
47,2022-12-06,1034.0,58.0,SUNPHARMA.NS,13.20,1.42,1.00,1052.0,737.0,94.14,5.86,1056.0,2.13
6,2022-12-06,1184.0,55.0,INDUSINDBK.NS,13.09,1.34,0.43,1264.0,778.0,83.56,16.44,1393.0,17.65
54,2022-12-06,2094.0,66.0,LT.NS,12.98,1.24,1.16,2104.0,1472.0,98.34,1.66,2036.0,-2.77
42,2022-12-06,337.0,42.0,ITC.NS,12.06,1.66,0.00,361.0,208.0,84.51,15.49,350.0,3.86
11,2022-12-06,1263.0,48.0,M&M.NS,11.68,1.24,0.16,1354.0,691.0,86.28,13.72,1468.0,16.23


In [14]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,668.0,40.0,MOTILALOFS.NS,-13.71,-1.55,-1.01,980.0,654.0,4.26,95.74,930.0,39.22
1,2022-12-06,465.0,73.0,GODREJIND.NS,-0.51,-0.64,1.15,644.0,405.0,24.94,75.06,638.0,37.20
2,2022-12-06,468.0,40.0,ICICIPRULI.NS,-11.20,-1.14,-1.00,616.0,447.0,12.43,87.57,640.0,36.75
3,2022-12-06,1285.0,60.0,SBILIFE.NS,6.38,0.50,0.56,1332.0,1025.0,84.72,15.28,1604.0,24.82
12,2022-12-06,587.0,68.0,HDFCLIFE.NS,3.31,0.35,1.44,694.0,504.0,43.58,56.42,681.0,16.01
14,2022-12-06,4053.0,56.0,NAUKRI.NS,-4.34,-0.64,0.33,5740.0,3375.0,28.67,71.33,4677.0,15.40
24,2022-12-06,1116.0,58.0,MUTHOOTFIN.NS,-2.38,-0.44,0.93,1540.0,956.0,27.37,72.63,1217.0,9.05
30,2022-12-06,1089.0,57.0,TECHM.NS,-6.30,-0.69,0.66,1806.0,965.0,14.72,85.28,1158.0,6.34
31,2022-12-06,1613.0,57.0,INFY.NS,3.13,0.38,1.22,1940.0,1365.0,43.11,56.89,1715.0,6.32
35,2022-12-06,3356.0,43.0,DIVISLAB.NS,-11.76,-1.24,-0.60,4678.0,3266.0,6.37,93.63,3560.0,6.08


In [15]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     19.794085
1     18.774604
2     18.547492
3     12.526496
12     8.080145
14     7.772282
24     4.567478
30     3.199758
31     3.189664
35     3.068537
48     0.661149
50     0.000000
51    -0.181690
Name: Upside%, dtype: float64

In [16]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-06,6699.0,37.0,BAJFINANCE.NS,-2.33,-0.20,-0.85,7853.0,5284.0,55.06,44.94,8015.0,19.64
5,2022-12-06,923.0,51.0,ICICIBANK.NS,11.43,1.33,0.66,953.0,654.0,89.85,10.15,1094.0,18.53
6,2022-12-06,1184.0,55.0,INDUSINDBK.NS,13.09,1.34,0.43,1264.0,778.0,83.56,16.44,1393.0,17.65
7,2022-12-06,905.0,68.0,AXISBANK.NS,17.95,1.69,1.42,915.0,626.0,96.42,3.58,1060.0,17.13
8,2022-12-06,3300.0,34.0,EICHERMOT.NS,5.15,0.86,-0.69,3850.0,2256.0,65.48,34.52,3865.0,17.12
9,2022-12-06,555.0,44.0,JUBLFOOD.NS,-6.36,-0.67,-1.05,786.0,464.0,28.35,71.65,649.0,16.94
10,2022-12-06,609.0,59.0,SBIN.NS,14.90,1.51,1.03,617.0,435.0,95.58,4.42,710.0,16.58
11,2022-12-06,1263.0,48.0,M&M.NS,11.68,1.24,0.16,1354.0,691.0,86.28,13.72,1468.0,16.23
13,2022-12-06,427.0,51.0,TATAMOTORS.NS,-2.04,-0.23,0.24,525.0,372.0,36.01,63.99,495.0,15.93
15,2022-12-06,78.0,60.0,NATIONALUM.NS,-5.54,-1.20,1.33,130.0,67.0,18.10,81.90,90.0,15.38


In [17]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
9,2022-12-06,555.0,44.0,JUBLFOOD.NS,-6.36,-0.67,-1.05,786.0,464.0,28.35,71.65,649.0,16.94
17,2022-12-06,2617.0,47.0,TITAN.NS,5.52,0.56,0.08,2770.0,1936.0,81.65,18.35,2989.0,14.21
25,2022-12-06,1222.0,45.0,HAVELLS.NS,-2.90,-0.16,-0.16,1418.0,1070.0,43.81,56.19,1328.0,8.67
29,2022-12-06,2655.0,61.0,HINDUNILVR.NS,7.95,0.57,0.70,2702.0,1944.0,93.74,6.26,2832.0,6.67
30,2022-12-06,1089.0,57.0,TECHM.NS,-6.30,-0.69,0.66,1806.0,965.0,14.72,85.28,1158.0,6.34
31,2022-12-06,1613.0,57.0,INFY.NS,3.13,0.38,1.22,1940.0,1365.0,43.11,56.89,1715.0,6.32
32,2022-12-06,3162.0,54.0,ASIANPAINT.NS,0.31,-0.03,-0.03,3576.0,2580.0,58.40,41.60,3361.0,6.29
35,2022-12-06,3356.0,43.0,DIVISLAB.NS,-11.76,-1.24,-0.60,4678.0,3266.0,6.37,93.63,3560.0,6.08
40,2022-12-06,618.0,54.0,BERGEPAINT.NS,-3.66,-0.47,0.16,791.0,561.0,25.00,75.00,651.0,5.34
42,2022-12-06,337.0,42.0,ITC.NS,12.06,1.66,0.00,361.0,208.0,84.51,15.49,350.0,3.86


In [18]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
60,2022-12-06,716.0,41.0,IRCTC.NS,-0.35,0.14,0.00,903.0,571.0,43.81,56.19,571.0,-20.25
45,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,0.62,4842.0,2742.0,32.95,67.05,3520.0,2.50
50,2022-12-06,3385.0,58.0,TCS.NS,1.88,0.24,1.01,4019.0,2982.0,38.90,61.10,3385.0,0.00
20,2022-12-06,4374.0,45.0,DRREDDY.NS,0.81,0.25,0.32,4907.0,3719.0,55.15,44.85,4829.0,10.40
49,2022-12-06,1586.0,48.0,COLPAL.NS,1.76,0.26,0.19,1678.0,1394.0,67.76,32.24,1606.0,1.26
36,2022-12-06,2239.0,58.0,DEEPAKNTR.NS,5.86,0.28,0.46,2660.0,1720.0,55.21,44.79,2371.0,5.90
12,2022-12-06,587.0,68.0,HDFCLIFE.NS,3.31,0.35,1.44,694.0,504.0,43.58,56.42,681.0,16.01
26,2022-12-06,886.0,67.0,GODREJCP.NS,4.33,0.35,0.58,968.0,665.0,72.85,27.15,958.0,8.13
38,2022-12-06,593.0,70.0,DABUR.NS,7.16,0.36,1.07,595.0,485.0,97.82,2.18,625.0,5.40
18,2022-12-06,1933.0,66.0,INDIGO.NS,4.53,0.38,1.03,2265.0,1549.0,53.61,46.39,2176.0,12.57
